## set up

In [5]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 

/Users/catherineyeh/.local/share/virtualenvs/story-viz-mNgXibik/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [10]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [11]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [12]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [13]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [423]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "emma" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "character" # character, theme, 2

In [424]:
# optional, but this part needs to be manually looked up
story_info = {
  "title": "Emma",
  "type": "Book",
  "author": "Jane Austen",
  "year": 1815,
  "url": "https://www.gutenberg.org/ebooks/158",
  "image": "https://cdn.kobo.com/book-images/1ea81b28-2fa1-4796-b9b0-710123b48227/1200/1200/False/emma-503.jpg",
}

In [425]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [426]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [427]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [428]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [429]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

270231
1


In [108]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

Chunk 0


In [109]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
num_chapters = sum([res.numChapters for res in all_res])
markers = []
for res in all_res:
    markers.extend(res.markers)

num_chapters, markers

(12,
 ['CHAPTER I.     Down the Rabbit-Hole',
  'CHAPTER II.    The Pool of Tears',
  'CHAPTER III.   A Caucus-Race and a Long Tale',
  'CHAPTER IV.    The Rabbit Sends in a Little Bill',
  'CHAPTER V.     Advice from a Caterpillar',
  'CHAPTER VI.    Pig and Pepper',
  'CHAPTER VII.   A Mad Tea-Party',
  'CHAPTER VIII.  The Queen’s Croquet-Ground',
  'CHAPTER IX.    The Mock Turtle’s Story',
  'CHAPTER X.     The Lobster Quadrille',
  'CHAPTER XI.    Who Stole the Tarts?',
  'CHAPTER XII.   Alice’s Evidence'])

In [110]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers
first_line, last_line

('Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, "and what is the use of a book," thought Alice "without pictures or conversations?"',
 'and how she would gather about her other little children, and make _their_ eyes bright and eager with many a strange tale, perhaps even with the dream of Wonderland of long ago: and how she would feel with all their simple sorrows, and find a pleasure in all their simple joys, remembering her own child-life, and the happy summer days.')

In [430]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [431]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [432]:
# # remove any marker that doesn't include one of these words
# exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [114]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}

if not os.path.exists(f"json/{og_story_name}"):
    os.makedirs(f"json/{og_story_name}")
with open(f"json/{og_story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [433]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [434]:
story_formatted[-100:] 

'harge, she had never before for a moment supposed it could make him unjust to the merit of another. '

In [435]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(1430,
 'Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world with very little to distress or vex her.')

In [29]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(34025,
 'II ',
 'd, and I was alone again in the unquiet darkness. II About halfway between West Egg and New York the')

In [25]:
last_ind = story_formatted.find(last_line)
last_ind, last_line

(268997,
 'So we beat on, boats against the current, borne back ceaselessly into the past.')

In [15]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    # print("marker", marker)
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    # print("next_marker", next_marker)
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line
    # print("formatted_last_line", formatted_last_line)

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.rfind(cur_first_line, cur_min_index)
    print("cur_first_line", repr(cur_first_line))
    print("first_line_index", first_line_index)
    print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    print("cur_last_line", repr(cur_last_line))
    print("last_line_index", last_line_index)
    print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
    
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

cur_first_line 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.'
first_line_index 1858
Snippet around first_line_index: 'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first enter'
cur_last_line 'CHAPTER II. '
last_line_index 6309
Snippet around last_line_index: 'standing, little information, and uncertain temper. When she was discontented she fancied herself nervous. The business of her life was to get her daughters married; its solace was visiting and news. '
cur_first_line 'CHAPTER II. '
first_line_index 452883
Snippet around first_line_index: 'CHAPTER II. Elizabeth had settled it that Mr. Darcy would bring his sister to visit her, the very day after her reaching Pemberley; and was consequently resolved not to be out of sight of the inn the '
cur_last_line 'CHAPTER III. '
la

In [16]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter CHAPTER I.
first_line: LINE 1: It is a truth universally acknowledged, that a single man in possession
LINE 2: of a good fo
last_line: he business of her life was to get her
LINE 80: daughters married; its solace was visiting and news.


Chapter CHAPTER II.
first_line: LINE 1: Elizabeth had settled it that Mr. Darcy would bring his sister to visit
LINE 2: her, the ver
last_line:  and a positive engagement made of his meeting
LINE 197: some of the gentlemen at Pemberley by noon.


Chapter CHAPTER III.
first_line: LINE 1: Convinced as Elizabeth now was that Miss Bingley's dislike of her had
LINE 2: originated in 
last_line: , and Mrs. Gardiner would have been highly gratified by
LINE 153: her niece's beginning the subject.


Chapter CHAPTER IV.
first_line: LINE 1: Elizabeth had been a good deal disappointed in not finding a letter from
LINE 2: Jane, on th
last_line: f time than she could
LINE 259: have supposed, seated in the carriage, and on the road to Longbourn.


Chapter CHAP

In [17]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{og_story_name}"):
    os.makedirs(f"chapters/{og_story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter CHAPTER I. saved.
Chapter CHAPTER II. saved.
Chapter CHAPTER III. saved.
Chapter CHAPTER IV. saved.
Chapter CHAPTER V. saved.
Chapter CHAPTER VI. saved.
Chapter CHAPTER VII. saved.
Chapter CHAPTER VIII. saved.
Chapter CHAPTER IX. saved.
Chapter CHAPTER X. saved.
Chapter CHAPTER XI. saved.
Chapter CHAPTER XII. saved.
Chapter CHAPTER XIII. saved.
Chapter CHAPTER XIV. saved.
Chapter CHAPTER XV. saved.
Chapter CHAPTER XVI. saved.
Chapter CHAPTER XVII. saved.
Chapter CHAPTER XVIII. saved.
Chapter CHAPTER XIX. saved.
Chapter CHAPTER XX. saved.
Chapter CHAPTER XXI. saved.
Chapter CHAPTER XXII. saved.
Chapter CHAPTER XXIII. saved.


In [436]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [437]:
chapters[0]

'LINE 1: Emma Woodhouse, handsome, clever, and rich, with a comfortable home and\nLINE 2: happy disposition, seemed to unite some of the best blessings of\nLINE 3: existence; and had lived nearly twenty-one years in the world with very\nLINE 4: little to distress or vex her.\nLINE 5: She was the youngest of the two daughters of a most affectionate,\nLINE 6: indulgent father; and had, in consequence of her sister\'s marriage,\nLINE 7: been mistress of his house from a very early period. Her mother had\nLINE 8: died too long ago for her to have more than an indistinct remembrance\nLINE 9: of her caresses; and her place had been supplied by an excellent woman\nLINE 10: as governess, who had fallen little short of a mother in affection.\nLINE 11: Sixteen years had Miss Taylor been in Mr. Woodhouse\'s family, less as a\nLINE 12: governess than a friend, very fond of both daughters, but particularly\nLINE 13: of Emma. Between _them_ it was more the intimacy of sisters. Even\nLINE 14: before 

## analyze scene

In [302]:
scene_type = "character" # location, character, or n/a

In [104]:
extra_scene_instruction = "Analyze the scenes in this chapter."
if scene_type == "location":
    extra_scene_instruction += " Start a new scene when the location changes."
elif scene_type == "character":
    extra_scene_instruction += " Start a new scene when new characters enter or old characters leave."

extra_scene_instruction += "\nChapter text:"

extra_scene_instruction

'Analyze the scenes in this chapter. Start a new scene when new characters enter or old characters leave.\nChapter text:'

In [105]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [106]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A made up quote that illustrates the character's emotions and/or actions if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A made up quote that illustrates the theme if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [107]:
def get_scene_info(chapters, chapter_num):
    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = llm3.invoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = scene_llm.invoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [110]:
def get_scene_details(scenes, formatted_chapter_scenes):
    all_scene_details = []
    for i, scene in enumerate(scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        if len(scene_text) == 0:
            all_scene_details.append([])
            continue
        # get scene details for each scene
        cur_scene = scene
        invalid_input = True
        # if model_type == 'gemini':
        #     prompt = get_scenedetails_prompt(cur_scene)
        #     scene_details = llm.invoke(prompt)
        #     print(scene_details)
        #     scene_details = format_gemini_response(scene_details)
        #     print(scene_details)
        #     scene_details = DotDict(scene_details)
        # else:

        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct quotes from the text.
                          Don't paraphrase or change the text in any way.
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a direct quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          make up a quote that fits the character's role and emotion in the scene.
                          Leave the "fake_quote" field empty if you can find a direct quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way.
                            Try your best to find a direct quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            make up a quote that fits the theme's role and emotion in the scene.
                            Leave the "fake_quote" field empty if you can find a direct quote.
                        
                            Scene text:

                            """

        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = scene_details_llm_theme.invoke(start_prompt + cur_scene)
                else:
                    scene_details = scene_details_llm.invoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except:
                invalid_input = True
        all_scene_details.append(scene_details)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if role is not N/A and emotion is not N/A and quote is not "No quote available"
            if not (character.role == "N/A" and character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [ ]:
# get scene info for each chapter
for i, marker in enumerate(markers):
    print(f"Chapter {marker} start.")
    scenes, formatted_chapter_scenes = get_scene_info(chapters, i)
    print(f"Chapter {marker} scenes done.")
    formatted_chapter_scenes = get_scene_details(scenes, formatted_chapter_scenes)
    print(f"Chapter {marker} details done.")

    # remove scenes with no text
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
    # remove characters with no quote
    for scene in formatted_chapter_scenes:
        scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
        # print(scene)
    
    # remove scenes with no characters
    formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(formatted_chapter_scenes, f, indent=4)

    print(f"Chapter {marker} done.\n")

Chapter CHAPTER I start.
FIRST LINE: 1
LAST LINE: 25

FIRST LINE: 26
LAST LINE: 34

FIRST LINE: 35
LAST LINE: 72

FIRST LINE: 73
LAST LINE: 138

FIRST LINE: 139
LAST LINE: 206

FIRST LINE: 207
LAST LINE: 281

Chapter CHAPTER I scenes done.
Chapter CHAPTER I details done.
Chapter CHAPTER I done.

Chapter CHAPTER II start.
FIRST LINE: 1
LAST LINE: 13

FIRST LINE: 14
LAST LINE: 25

FIRST LINE: 29
LAST LINE: 42

FIRST LINE: 43
LAST LINE: 79

FIRST LINE: 80
LAST LINE: 89

FIRST LINE: 90
LAST LINE: 105

FIRST LINE: 106
LAST LINE: 125

FIRST LINE: 126
LAST LINE: 147

Chapter CHAPTER II scenes done.
Chapter CHAPTER II details done.
Chapter CHAPTER II done.

Chapter CHAPTER III start.
FIRST LINE: 1
LAST LINE: 15

FIRST LINE: 16
LAST LINE: 20

FIRST LINE: 21
LAST LINE: 70

FIRST LINE: 71
LAST LINE: 78

FIRST LINE: 79
LAST LINE: 85

FIRST LINE: 86
LAST LINE: 120

FIRST LINE: 123
LAST LINE: 138

FIRST LINE: 139
LAST LINE: 161

Chapter CHAPTER III scenes done.
Chapter CHAPTER III details done.
Chap

In [ ]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

CHAPTER I. Mrs. Rachel Lynde is Surprised
CHAPTER II. Matthew Cuthbert is surprised
CHAPTER III. Marilla Cuthbert is Surprised
CHAPTER IV. Morning at Green Gables
CHAPTER V. Anne's History
CHAPTER VI. Marilla Makes Up Her Mind
CHAPTER VII. Anne Says Her Prayers
CHAPTER VIII. Anne's Bringing-up Is Begun
CHAPTER IX. Mrs. Rachel Lynde Is Properly Horrified
CHAPTER X. Anne's Apology
CHAPTER XI. Anne's Impressions of Sunday-School
CHAPTER XII. A Solemn Vow and Promise
CHAPTER XIII. The Delights of Anticipation
CHAPTER XIV. Anne's Confession
CHAPTER XV. A Tempest in the School Teapot
CHAPTER XVI. Diana Is Invited to Tea with Tragic Results
CHAPTER XVII. A New Interest in Life
CHAPTER XVIII. Anne to the Rescue
CHAPTER XIX. A Concert a Catastrophe and a Confession
CHAPTER XX. A Good Imagination Gone Wrong
CHAPTER XXII. Anne is Invited Out to Tea
CHAPTER XXIII. Anne Comes to Grief in an Affair of Honor
CHAPTER XXIV. Miss Stacy and Her Pupils Get Up a Concert
CHAPTER XXV. Matthew Insists on Puff

In [368]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [369]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [370]:
len(all_characters), len(all_locations)

(471, 180)

In [371]:
all_characters

{'Curiosity': "Mrs. Rachel's desire to know Matthew's whereabouts",
 'Community Oversight': "Mrs. Rachel's role in monitoring the community",
 'Tradition': 'The established norms of Avonlea life',
 'Isolation': 'Highlights the seclusion of Green Gables and its inhabitants.',
 'Cleanliness': "Emphasizes the meticulous nature of Marilla's housekeeping.",
 'Mystery': 'Creates intrigue about the Cuthberts and their unusual behavior.',
 'Friendship': 'The bond between Marilla and Rachel highlights their connection despite differences.',
 'Change': "The decision to adopt a boy signifies a major shift in Marilla and Matthew's lives.",
 'Community': "Mrs. Rachel's reaction reflects the close-knit nature of the Avonlea community and its gossip.",
 'Caution': 'Mrs. Rachel warns Marilla about the risks of adopting a child.',
 'Parental Responsibility': 'Marilla reflects on the responsibilities of adopting a child.',
 'Community Judgment': 'Mrs. Rachel represents societal opinions on adoption.',
 

In [372]:
all_locations

["Lynde's Hollow",
 'Green Gables Kitchen',
 "Marilla's home in Avonlea",
 'Green Gables',
 'Bright River station',
 'On the road to Bright River',
 'Bright River Station',
 'On the way to the Island',
 'Avenue of Apple Trees',
 'The Avenue, Green Gables',
 "Cuthbert's home entrance",
 "Child's emotional outburst",
 'A conversation setting',
 "Anne's Supper Table",
 "Anne's new room",
 "Marilla's kitchen",
 "Matthew and Marilla's home",
 "Anne's bedroom at Green Gables",
 'Green Gables bedroom',
 'Dining Room',
 "Matthew's home",
 'Green Gables kitchen',
 "Marilla's home",
 'On a drive',
 'Shore Road',
 "Anne's Childhood Memories",
 "Anne's conversation with Marilla",
 'Woodsy Shore Road',
 'White Sands Hotel',
 "Mrs. Spencer's house",
 "Mrs. Spencer's parlor",
 "Anne's bedroom",
 'Sitting room',
 'Orchard Slope',
 "Anne's Room",
 'Cherry Blossoms Garden',
 'Apple Orchard and Woods',
 'Orchard at sunset',
 'Kitchen of Mrs. Rachel',
 'Cuthbert household',
 'East Gable',
 "Matthew's hous

In [373]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [374]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [375]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [ ]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [378]:
output

AIMessage(content='{\n  "Curiosity": ["Mrs. Rachel\'s desire to know Matthew\'s whereabouts"],\n  "Community": ["Community Oversight", "Community Judgment", "Community Gossip", "Community and Belonging", "Community Connection"],\n  "Tradition": ["Tradition", "Tradition vs. Change"],\n  "Isolation": ["Isolation", "Isolation in Grief", "Isolation in Sorrow"],\n  "Cleanliness": ["Cleanliness", "Neatness"],\n  "Mystery": ["Mystery", "Mystery of Loss"],\n  "Friendship": ["Friendship", "Friendship and Loyalty", "Friendship Anxiety", "Friendship Dynamics", "Friendship Support", "Friendship Strain", "Friendship Excitement", "Friendship and Hospitality"],\n  "Change": ["Change", "Fear of Change"],\n  "Caution": ["Caution"],\n  "Parental Responsibility": ["Parental Responsibility", "Parental Roles", "Parental Authority", "Parental Approval", "Parental Expectations"],\n  "Fear": ["Fear of Change", "Fear and Surprise", "Fear of the Unknown", "Fear of Rain", "Fear and Panic", "Fear of Loss", "Fear 

In [379]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [380]:
output

{'Curiosity': ["Mrs. Rachel's desire to know Matthew's whereabouts"],
 'Community': ['Community Oversight',
  'Community Judgment',
  'Community Gossip',
  'Community and Belonging',
  'Community Connection'],
 'Tradition': ['Tradition', 'Tradition vs. Change'],
 'Isolation': ['Isolation', 'Isolation in Grief', 'Isolation in Sorrow'],
 'Cleanliness': ['Cleanliness', 'Neatness'],
 'Mystery': ['Mystery', 'Mystery of Loss'],
 'Friendship': ['Friendship',
  'Friendship and Loyalty',
  'Friendship Anxiety',
  'Friendship Dynamics',
  'Friendship Support',
  'Friendship Strain',
  'Friendship Excitement',
  'Friendship and Hospitality'],
 'Change': ['Imminent Change'],
 'Caution': ['Caution'],
 'Parental Responsibility': ['Parental Responsibility',
  'Parental Roles',
  'Parental Authority',
  'Parental Approval',
  'Parental Expectations'],
 'Fear': ['Fear of Change',
  'Fear and Surprise',
  'Fear of the Unknown',
  'Fear of Rain',
  'Fear and Panic',
  'Fear of Loss',
  'Fear and Relief',

In [381]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [382]:
location_output

AIMessage(content='{\n    "Green Gables": ["Green Gables Kitchen", "Green Gables", "The Avenue, Green Gables", "Cuthbert\'s home entrance", "Anne\'s Supper Table", "Anne\'s new room", "Marilla\'s kitchen", "Matthew and Marilla\'s home", "Anne\'s bedroom at Green Gables", "Green Gables bedroom", "Dining Room", "Matthew\'s home", "Green Gables kitchen", "Marilla\'s home", "East Gable", "Matthew\'s house upstairs", "Marilla\'s Hallway", "Green Gables Lane", "Gable room", "Green Gables garden", "West window area", "Anne\'s home", "Anne\'s sewing area", "Lover\'s Lane at Green Gables", "Green Gables in October", "Anne\'s House", "The orchard", "Room pantry", "Anne\'s kitchen window", "Cheerful kitchen at Green Gables", "Anne\'s home kitchen", "Cuthbert\'s home", "Anne\'s home and orchard", "Anne\'s gable window", "Anne\'s Kitchen Table", "Green Gables, August evening", "Anne\'s home before tea", "Anne\'s Kitchen Door", "Marilla\'s living room", "Anne\'s room in Avonlea", "Matthew\'s kitchen

In [383]:
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [384]:
location_output

{'Green Gables': ['Green Gables Kitchen',
  'Green Gables',
  'The Avenue, Green Gables',
  "Cuthbert's home entrance",
  "Anne's Supper Table",
  "Anne's new room",
  "Marilla's kitchen",
  "Matthew and Marilla's home",
  "Anne's bedroom at Green Gables",
  'Green Gables bedroom',
  'Dining Room',
  "Matthew's home",
  'Green Gables kitchen',
  "Marilla's home",
  'East Gable',
  "Matthew's house upstairs",
  "Marilla's Hallway",
  'Green Gables Lane',
  'Gable room',
  'Green Gables garden',
  'West window area',
  "Anne's home",
  "Anne's sewing area",
  "Lover's Lane at Green Gables",
  'Green Gables in October',
  "Anne's House",
  'The orchard',
  'Room pantry',
  "Anne's kitchen window",
  'Cheerful kitchen at Green Gables',
  "Anne's home kitchen",
  "Cuthbert's home",
  "Anne's home and orchard",
  "Anne's gable window",
  "Anne's Kitchen Table",
  'Green Gables, August evening',
  "Anne's home before tea",
  "Anne's Kitchen Door",
  "Marilla's living room",
  "Anne's room in 

In [385]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [386]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [387]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [388]:
len(all_characters_new), len(all_locations_new)

(239, 64)

In [389]:
all_characters_new 

{'Curiosity': 5,
 'Community': 9,
 'Tradition': 9,
 'Isolation': 21,
 'Cleanliness': 3,
 'Mystery': 2,
 'Friendship': 88,
 'Change': 3,
 'Caution': 1,
 'Parental Responsibility': 12,
 'Fear': 14,
 'Judgment': 14,
 'Nature': 32,
 'Social Anxiety': 3,
 'Unexpected Arrival': 2,
 'Miscommunication': 2,
 'Imagination': 40,
 'Childhood': 34,
 'Misunderstanding': 2,
 'Connection': 3,
 'Belonging': 8,
 'Hope': 11,
 'Self-acceptance': 6,
 'Homecoming': 2,
 'Anticipation': 10,
 'Identity': 7,
 'Gender': 4,
 'Rejection': 3,
 'Despair': 3,
 'Acceptance': 5,
 'Food': 1,
 'Emotional Struggle': 3,
 'Support': 13,
 'Perturbation': 1,
 'Compassion Vs. Duty': 1,
 'Communication': 3,
 'Desire For Connection': 1,
 'Childcare': 1,
 'Sympathy': 3,
 'Skepticism': 2,
 'Resignation': 1,
 'Decision Making': 1,
 'Positive Outlook': 1,
 'Buried Hopes': 1,
 'Romanticism': 9,
 'Loss': 10,
 'Social Status And Displacement': 1,
 'Education': 6,
 'Love For Poetry': 1,
 'Empathy': 2,
 'Parenthood': 1,
 'Perception': 1,

In [390]:
all_locations_new

["Lynde's Hollow",
 'Green Gables',
 'Bright River',
 'Prince Edward Island',
 'Avenue Of Apple Trees',
 "Child's Emotional Outburst",
 'A Conversation Setting',
 'On A Drive',
 'Shore Road',
 "Anne's Childhood Memories",
 "Anne's Conversation With Marilla",
 'White Sands Hotel',
 "Mrs. Spencer's House",
 "Anne's Bedroom",
 'Sitting Room',
 'Orchard Slope',
 "Mrs. Rachel's Home",
 'Cuthbert Household',
 "Mrs. Lynde's Home",
 'Church',
 'Avonlea',
 "Barry's House",
 'Idlewild Playhouse',
 "Marilla's Room",
 'Picnic Ground Preparation',
 "Lover's Lane",
 "Anne's First Day At School",
 'Autumn Road',
 'East Window At Twilight',
 'Charlottetown',
 'The Audience Event',
 "Miss Barry's House",
 'Violet Vale',
 'Dim Path Beyond',
 'Orchard With Apples',
 'School In October',
 'Back Yard Under Moon',
 'Carmody',
 'Hollow By The Bridge',
 'Birch Path',
 "Anne's Story Club",
 "Anne's Discussion With Marilla",
 'Walking Home In Spring',
 'Pond',
 'Woodland Setting',
 'Schoolyard',
 'Path To The P

## produce chapter summaries

In [391]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [392]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [ ]:
# create summary json of each chapter
chapter_summaries = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
        
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()

    # get summary of chapter
    scene_summaries = " ".join([scene["summary"] for scene in scenes])
    summary = chapter_llm.invoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter),
        "num_lines": len(chapter.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }
    chapter_summaries.append(chapter_summary)
    
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
    for scene in scenes:
        chapter = scene["chapter"]
        chapter_summary = chapter_summaries[markers.index(chapter)]
        # add location to chapter summary
        location = scene["location"]
        chapter_summary_locations = chapter_summary["locations"]
        if location not in chapter_summary_locations:
            chapter_summary_locations[location] = 1
        else:
            chapter_summary_locations[location] += 1
        # add importance to chapter summary
        importance = scene["importance"]
        chapter_summary["importance"].append(importance)
        # add conflict to chapter summary
        conflict = scene["conflict"]
        chapter_summary["conflict"].append(conflict)
        # add characters to chapter summary
        characters = scene["characters"]
        for character in characters:
            char = character["name"]
            chapter_summary_characters = chapter_summary["characters"]
            if char not in chapter_summary_characters:
                chapter_summary_characters[char] = 1
            else:
                chapter_summary_characters[char] += 1

# now average importance and conflict for each chapter
for chapter_summary in chapter_summaries:
    if len(chapter_summary["importance"]) == 0:
       print(f"Chapter {chapter_summary['chapter']} has no scenes.")
    chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
    chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

# rank each chapter by importance and conflict
importances = []
conflicts = []
for chapter_summary in chapter_summaries:
    importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
    conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
    
# sort importances
sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
# add importance_rank to each chapter
for k, (j, _) in enumerate(sorted_importances):
    chapter_summaries[markers.index(j)]["importance_rank"] = k+1

# sort conflicts
sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
# add conflict_rank to each chapter
for k, (j, _) in enumerate(sorted_conflicts):
    chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

In [396]:
chapter_summaries

[{'chapter': 'CHAPTER I. Mrs. Rachel Lynde is Surprised',
  'scenes': 5,
  'length': 20816,
  'num_lines': 297,
  'summary': "Mrs. Rachel Lynde visits Marilla at Green Gables and expresses her concerns about Marilla's plan to adopt a boy.",
  'importance': 0.82,
  'conflict': 0.36,
  'locations': {"Lynde's Hollow": 1, 'Green Gables': 3, 'Bright River': 1},
  'characters': {'Curiosity': 1,
   'Community': 3,
   'Tradition': 2,
   'Isolation': 2,
   'Cleanliness': 1,
   'Mystery': 1,
   'Friendship': 1,
   'Change': 1,
   'Caution': 1,
   'Parental Responsibility': 1,
   'Fear': 1,
   'Judgment': 1},
  'importance_rank': 19,
  'conflict_rank': 16},
 {'chapter': 'CHAPTER II. Matthew Cuthbert is surprised',
  'scenes': 6,
  'length': 30347,
  'num_lines': 410,
  'summary': 'Matthew Cuthbert unexpectedly meets Anne, a spirited girl, instead of the boy he intended to adopt, and decides to take her home to Green Gables.',
  'importance': 0.83,
  'conflict': 0.33,
  'locations': {'Bright River

In [397]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [398]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [399]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Friendship': 88,
 'Imagination': 40,
 'Childhood': 34,
 'Nature': 32,
 'Nostalgia': 22,
 'Isolation': 21,
 'Growth': 18,
 'Fear': 14,
 'Judgment': 14,
 'Support': 13,
 'Parental Responsibility': 12,
 'Hope': 11,
 'Competition': 11,
 'Anticipation': 10,
 'Loss': 10,
 'Ambitions': 10,
 'Community': 9,
 'Tradition': 9,
 'Romanticism': 9,
 'Pride': 9}

In [400]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
20
20
20
20
20
20
20
19


In [401]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [ ]:
all_res = []
used_colors = []
error_chunks = []
for i, arr in enumerate(character_arr):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        # filter all_json to only include characters in arr
        new_scenes = []
        # for scene in all_json["scenes"]:
        for scene in scenes:
            include = False
            for character in scene["characters"]:
                if character["name"] in arr:
                    include = True
                    break
            if include:
                new_scenes.append(scene)

        # take middle 126000 characters of new_scenes
        new_scenes_str = json.dumps(new_scenes)
        new_scenes_str = new_scenes_str[:126000]

        try:
            # prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list. Make sure each character is assigned a unique color."
            prompt = f"characters: {arr}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(arr)} characters as in the original list."
            # if len(used_colors) > 0:
            #     prompt += f"\ndon't use any of these colors: {used_colors}"
            res = characterlist_llm.invoke(prompt)
            all_res.append(res)
            # int_list = res.characterlist
            # for j, char in enumerate(int_list):
            #     used_colors.append(char.color)
            print(f"Chunk {i} done.")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
            # error_chunks.append(i)
            # invalid_input = False


Chunk 0 done.
Chunk 1 done.
Chunk 2 done.
Chunk 3 done.
Chunk 4 done.
Chunk 5 done.
Chunk 6 done.
Chunk 7 done.
Chunk 8 done.
Chunk 9 done.
Chunk 10 done.
Chunk 11 done.


In [403]:
error_chunks

[]

In [404]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
20
20
20
20
20
20
20
19


239

In [405]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [406]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [407]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(45,
 ['Insecurity',
  'Social Reputation',
  'Relationships',
  'Self-Reflection',
  'Emotional Themes',
  'Change',
  'Career Aspirations',
  'Desire',
  'Emotional Worth',
  'Dedication',
  'Stress',
  'Nature',
  'Expression',
  'Reality',
  'Health',
  'Responsibility',
  'Social Dynamics',
  'Religion',
  'Family Connection',
  'Spiritual Growth',
  'Personal Growth',
  'Unromantic Reality',
  'Social Class',
  'Mystery',
  'Academic Themes',
  'Personal Development',
  'Health Themes',
  'Identity',
  'Transitions',
  'Social Themes',
  'Happiness',
  'Conflict',
  'Emotions',
  'Trust',
  'Emotional Pain',
  'Success',
  'Recognition',
  'Family',
  'Imagination',
  'Desires',
  'Support Themes',
  'Determination',
  'Education',
  'Urgency',
  'Self-identity'])

In [408]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [409]:
output

AIMessage(content='{\n  "Insecurity": ["Insecurity"],\n  "Social Reputation": ["Social Reputation", "Social Dynamics", "Social Class", "Social Themes"],\n  "Relationships": ["Relationships", "Family Connection", "Family"],\n  "Self-Reflection": ["Self-Reflection", "Self-identity"],\n  "Emotional Themes": ["Emotional Themes", "Emotional Worth", "Emotions", "Emotional Pain"],\n  "Change": ["Change", "Transitions"],\n  "Career Aspirations": ["Career Aspirations"],\n  "Desire": ["Desire", "Desires"],\n  "Dedication": ["Dedication", "Determination"],\n  "Stress": ["Stress"],\n  "Nature": ["Nature"],\n  "Expression": ["Expression"],\n  "Reality": ["Reality", "Unromantic Reality"],\n  "Health": ["Health", "Health Themes"],\n  "Responsibility": ["Responsibility"],\n  "Religion": ["Religion"],\n  "Spiritual Growth": ["Spiritual Growth"],\n  "Personal Growth": ["Personal Growth", "Personal Development"],\n  "Mystery": ["Mystery"],\n  "Academic Themes": ["Academic Themes", "Education"],\n  "Ident

In [410]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(29,
 {'Insecurity': ['Insecurity'],
  'Social Reputation': ['Social Reputation',
   'Social Dynamics',
   'Social Class',
   'Social Themes'],
  'Relationships': ['Relationships', 'Family Connection', 'Family'],
  'Self-Reflection': ['Self-Reflection', 'Self-identity'],
  'Emotional Themes': ['Emotional Themes',
   'Emotional Worth',
   'Emotions',
   'Emotional Pain'],
  'Change': ['Change', 'Transitions'],
  'Career Aspirations': ['Career Aspirations'],
  'Desire': ['Desire', 'Desires'],
  'Dedication': ['Dedication', 'Determination'],
  'Stress': ['Stress'],
  'Nature': ['Nature'],
  'Expression': ['Expression'],
  'Reality': ['Reality', 'Unromantic Reality'],
  'Health': ['Health', 'Health Themes'],
  'Responsibility': ['Responsibility'],
  'Religion': ['Religion'],
  'Spiritual Growth': ['Spiritual Growth'],
  'Personal Growth': ['Personal Growth', 'Personal Development'],
  'Mystery': ['Mystery'],
  'Academic Themes': ['Academic Themes', 'Education'],
  'Identity': ['Identity'],

In [411]:
# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [412]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [413]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

In [414]:
colorinfo = {}
for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Friendship': {'color': 'rgb(255, 182, 193)',
  'explanation': 'This soft pink represents warmth and connection, embodying the essence of friendship.'},
 'Imagination': {'color': 'rgb(135, 206, 250)',
  'explanation': 'A light sky blue symbolizes creativity and the boundless nature of imagination.'},
 'Childhood': {'color': 'rgb(255, 228, 196)',
  'explanation': 'This warm peach color reflects the innocence and joy of childhood.'},
 'Nature': {'color': 'rgb(60, 179, 113)',
  'explanation': 'A vibrant green represents the beauty and vitality of nature.'},
 'Nostalgia': {'color': 'rgb(255, 218, 185)',
  'explanation': 'A soft, warm beige evokes feelings of nostalgia and cherished memories.'},
 'Isolation': {'color': 'rgb(176, 224, 230)',
  'explanation': 'A pale blue conveys feelings of loneliness and solitude.'},
 'Growth': {'color': 'rgb(144, 238, 144)',
  'explanation': 'A fresh green symbolizes growth, renewal, and new beginnings.'},
 'Fear': {'color': 'rgb(255, 99, 71)',
  'explana

In [415]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

239


In [416]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(239,
 [{'name': 'Curiosity',
   'quote': "I'm clean puzzled, that's what, and I won't know a minute's peace of mind or conscience until I know what has taken Matthew Cuthbert out of Avonlea today.",
   'group': 'Emotional Themes',
   'color': '',
   'explanation': ''},
  {'name': 'Community',
   'quote': '...if she noticed anything odd or out of place she would never rest until she had ferreted out the whys and wherefores thereof.',
   'group': 'Social Reputation',
   'color': 'rgb(255, 140, 0)',
   'explanation': 'An earthy orange represents the warmth and connection of community.'},
  {'name': 'Tradition',
   'quote': '...he was the shyest man alive and hated to have to go among strangers or to any place where he might have to talk.',
   'group': 'Social Reputation',
   'color': 'rgb(139, 69, 19)',
   'explanation': 'A rich brown signifies the strength and stability of tradition.'},
  {'name': 'Isolation',
   'quote': "It's just _staying_, that's what.",
   'group': 'Emotional Theme

In [417]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(239,
 [{'name': 'Curiosity',
   'quote': "I'm clean puzzled, that's what, and I won't know a minute's peace of mind or conscience until I know what has taken Matthew Cuthbert out of Avonlea today.",
   'group': 'Emotional Themes',
   'color': '',
   'explanation': ''},
  {'name': 'Community',
   'quote': '...if she noticed anything odd or out of place she would never rest until she had ferreted out the whys and wherefores thereof.',
   'group': 'Social Reputation',
   'color': 'rgb(255, 140, 0)',
   'explanation': 'An earthy orange represents the warmth and connection of community.'},
  {'name': 'Tradition',
   'quote': '...he was the shyest man alive and hated to have to go among strangers or to any place where he might have to talk.',
   'group': 'Social Reputation',
   'color': 'rgb(139, 69, 19)',
   'explanation': 'A rich brown signifies the strength and stability of tradition.'},
  {'name': 'Isolation',
   'quote': "It's just _staying_, that's what.",
   'group': 'Emotional Theme

## produce location summaries

In [418]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [419]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [420]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

64
[{'name': "Lynde's Hollow", 'quote': "MRS. Rachel Lynde lived just where the Avonlea main road dipped down into a little hollow, fringed with alders and ladies' eardrops and traversed by a brook that had its source away back in the woods of the old Cuthbert place.", 'emoji': '🌳'}, {'name': 'Green Gables', 'quote': "The big, rambling, orchard-embowered house where the Cuthberts lived was a scant quarter of a mile up the road from Lynde's Hollow.", 'emoji': '🏡'}, {'name': 'Bright River', 'quote': 'When he reached Bright River there was no sign of any train; he thought he was too early, so he tied his horse in the yard of the small Bright River hotel.', 'emoji': '🌊'}, {'name': 'Prince Edward Island', 'quote': 'It was a pretty road, running along between snug farmsteads, with now and again a bit of balsamy fir wood to drive through or a hollow where wild plums hung out their filmy bloom.', 'emoji': '🇨🇦'}, {'name': 'Avenue Of Apple Trees', 'quote': 'The "Avenue," so called by the Newbrid

## generating the final json file!

In [421]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Anne of Green Gables',
 'type': 'Book',
 'author': 'L. M. Montgomery',
 'year': '1908',
 'url': 'https://www.gutenberg.org/ebooks/45',
 'image': 'https://upload.wikimedia.org/wikipedia/commons/8/8b/Montgomery_Anne_of_Green_Gables.jpg',
 'num_chapters': 36,
 'num_scenes': 258,
 'num_characters': 239,
 'num_locations': 64,
 'chapters': [{'chapter': 'CHAPTER I. Mrs. Rachel Lynde is Surprised',
   'scenes': 5,
   'length': 20816,
   'num_lines': 297,
   'summary': "Mrs. Rachel Lynde visits Marilla at Green Gables and expresses her concerns about Marilla's plan to adopt a boy.",
   'importance': 0.82,
   'conflict': 0.36,
   'locations': {"Lynde's Hollow": 1, 'Green Gables': 3, 'Bright River': 1},
   'characters': {'Curiosity': 1,
    'Community': 3,
    'Tradition': 2,
    'Isolation': 2,
    'Cleanliness': 1,
    'Mystery': 1,
    'Friendship': 1,
    'Change': 1,
    'Caution': 1,
    'Parental Responsibility': 1,
    'Fear': 1,
    'Judgment': 1},
   'importance_rank': 19,
  

In [422]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)